<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# `GridSearch` with `keras`

_Authors: Riley Dallas (ATX)_

---


### Learning Objectives

- Learn how to use `keras`'s `scikit_learn` wrappers.
- Use `GridSearchCV` to tune a `keras` model

In [1]:
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# Regression
---

`make_regression` is an excellent library for generating a random regression problem. This helps us focus on the model without having to worry about the dataset. 

In the cell below, use `make_regression` to generate 10,000 samples using 20 features.

In [2]:
X, y = make_regression(n_samples=10_000, n_features=20, random_state=42)

### Train/Test Split
---

We always want to have a validation set to test our model. Use the `train_test_split` function to split our `X` and `y` variables into a training set and a holdout set.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### `StandardScaler`
---

You want to scale your data for *any* model that uses Gradient Descent, which includes Neural Networks.

In [4]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

### Create your network topology
---

We'll create a neural network like we've done before, only this time we'll wrap the entire model in a function. Once we have that set up, we can use the `KerasRegressor` wrapper to set it up as an `sklearn` model, which we'll then apply `GridSearchCV`.

In [5]:
def model_func(layer_one_neurons=32, layer_two_neurons=32):
    model = Sequential()
    
    model.add(Dense(layer_one_neurons, activation='relu', input_shape=(20,)))
    
    model.add(Dense(layer_two_neurons, activation='relu'))
    
    model.add(Dense(1, activation=None))
    
    model.compile(loss='mse', optimizer='adam')
    
    return model

nn = KerasRegressor(build_fn=model_func, batch_size=512, verbose=0)

params = {
    'epochs': [1000],
    'layer_one_neurons': [32],
    'layer_two_neurons': [32]
}

gs = GridSearchCV(nn, param_grid=params, cv=2)
gs.fit(X_train_sc, y_train)
print(gs.best_score_)
gs.best_params_

-0.3272392749786377


{'epochs': 1000, 'layer_one_neurons': 32, 'layer_two_neurons': 32}

In [6]:
pred = gs.predict(X_test_sc)

In [7]:
metrics.r2_score(y_test, pred)

0.9999997934358322